In [97]:
import sys
!{sys.executable} -m pip install efficient_apriori

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [98]:
import pandas as pd
import numpy as np
import glob as gl
import pickle
from efficient_apriori import apriori
import warnings
import os
import pathlib
warnings.filterwarnings('ignore')

In [99]:
sub_MR1 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs_controled_inputs/FinalLog_controledInputs_sub_MR1.csv', index_col= 0)
sub_MR2 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs_controled_inputs/FinalLog_controledInputs_sub_MR2.csv', index_col= 0)
sub_MR3 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs_controled_inputs/FinalLog_controledInputs_sub_MR3.csv', index_col= 0)
sub_MR4 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs_controled_inputs/FinalLog_controledInputs_sub_MR4.csv', index_col= 0)

In [100]:
sub_MR1

,"(a,b)",input_a,input_b,const,output,MR1_input_a,MR1_input_b,MR1_inputs,MR1_output,MR1_checker
0,"(0,0)",0,0,6,0.0,0.0,0.0,"(0,0)",0.0,No-violated
1,"(1,0)",1,0,6,1.0,0.0,1.0,"(0,1)",-1.0,Violated
2,"(2,0)",2,0,6,2.0,0.0,2.0,"(0,2)",-2.0,Violated
3,"(3,0)",3,0,6,3.0,0.0,3.0,"(0,3)",-3.0,Violated
4,"(4,0)",4,0,6,4.0,0.0,4.0,"(0,4)",-4.0,Violated
...,...,...,...,...,...,...,...,...,...,...
95,"(5,9)",5,9,6,-4.0,9.0,5.0,"(9,5)",4.0,Violated
96,"(6,9)",6,9,6,-3.0,9.0,6.0,"(9,6)",3.0,Violated
97,"(7,9)",7,9,6,-2.0,9.0,7.0,"(9,7)",2.0,Violated
98,"(8,9)",8,9,6,-1.0,9.0,8.0,"(9,8)",1.0,Violated


In [101]:
def prepro(data):
    MR_name = list(sub_MR1.keys())[-1]

    for index, row in data.iterrows():
        auxR1 = row[MR_name]
        data.at[index, MR_name] = MR_name + ' == ' + auxR1
        data.at[index, 'input_a'] =  'input_a == ' + str(row['input_a'])
        data.at[index, 'input_b'] = 'input_b == ' + str(row['input_b'])
        data.at[index, 'output'] = 'output == ' + str(row['output'])
        data.at[index, 'MR1_input_a'] =  'MR1_input_a == ' + str(row['MR1_input_a'])
        data.at[index, 'MR1_input_b'] = 'MR1_input_b == ' + str(row['MR1_input_b'])
        data.at[index, 'MR1_inputs'] = 'MR1_inputs == ' + str(row['MR1_inputs'])
        data.at[index, 'MR1_output'] = 'MR1_output == ' + str(row['MR1_output'])

    return data

In [102]:
data = prepro(sub_MR1)
data1 = data[['(a,b)', 'input_a', 'input_b', 'output', 'MR1_input_a', 'MR1_input_b', 'MR1_inputs', 'MR1_output','MR1_checker']]
data2 = data[['input_a', 'input_b', 'output', 'MR1_input_a', 'MR1_input_b', 'MR1_output','MR1_checker']]
data3 = data[['input_a', 'input_b', 'output', 'MR1_input_a', 'MR1_input_b','MR1_checker']]
data4 = data[['input_a', 'input_b', 'MR1_input_a', 'MR1_input_b', 'MR1_checker']]
# data = data[["(a,b)",'input_a','input_b','MR1_checker']]

# data_2 = prepro(data_2)
# data_3 = prepro(data_3)
# data_4 = prepro(data_4)
# data_5 = prepro(data_5)

In [103]:
data

,"(a,b)",input_a,input_b,const,output,MR1_input_a,MR1_input_b,MR1_inputs,MR1_output,MR1_checker
0,"(0,0)",input_a == 0,input_b == 0,6,output == 0.0,MR1_input_a == 0.0,MR1_input_b == 0.0,"MR1_inputs == (0,0)",MR1_output == 0.0,MR1_checker == No-violated
1,"(1,0)",input_a == 1,input_b == 0,6,output == 1.0,MR1_input_a == 0.0,MR1_input_b == 1.0,"MR1_inputs == (0,1)",MR1_output == -1.0,MR1_checker == Violated
2,"(2,0)",input_a == 2,input_b == 0,6,output == 2.0,MR1_input_a == 0.0,MR1_input_b == 2.0,"MR1_inputs == (0,2)",MR1_output == -2.0,MR1_checker == Violated
3,"(3,0)",input_a == 3,input_b == 0,6,output == 3.0,MR1_input_a == 0.0,MR1_input_b == 3.0,"MR1_inputs == (0,3)",MR1_output == -3.0,MR1_checker == Violated
4,"(4,0)",input_a == 4,input_b == 0,6,output == 4.0,MR1_input_a == 0.0,MR1_input_b == 4.0,"MR1_inputs == (0,4)",MR1_output == -4.0,MR1_checker == Violated
...,...,...,...,...,...,...,...,...,...,...
95,"(5,9)",input_a == 5,input_b == 9,6,output == -4.0,MR1_input_a == 9.0,MR1_input_b == 5.0,"MR1_inputs == (9,5)",MR1_output == 4.0,MR1_checker == Violated
96,"(6,9)",input_a == 6,input_b == 9,6,output == -3.0,MR1_input_a == 9.0,MR1_input_b == 6.0,"MR1_inputs == (9,6)",MR1_output == 3.0,MR1_checker == Violated
97,"(7,9)",input_a == 7,input_b == 9,6,output == -2.0,MR1_input_a == 9.0,MR1_input_b == 7.0,"MR1_inputs == (9,7)",MR1_output == 2.0,MR1_checker == Violated
98,"(8,9)",input_a == 8,input_b == 9,6,output == -1.0,MR1_input_a == 9.0,MR1_input_b == 8.0,"MR1_inputs == (9,8)",MR1_output == 1.0,MR1_checker == Violated


In [104]:
def ruleGenerator(transactions_df, support, conf):
    """
        Generate set of rules with given support and confidence using transactions_df
    """
    # print(transactions_df.head(20))
    transactions_array = transactions_df.to_numpy()
    transactions_list = transactions_array.tolist()
    itemSets, rules = apriori(transactions_list, min_support=support, min_confidence=conf)
    return itemSets, rules


In [105]:
items1, rules1 = ruleGenerator(data1, 0.1, float(1))
items2, rules2 = ruleGenerator(data2, 0.1, float(1))
items3, rules3 = ruleGenerator(data3, 0.1, float(1))
items4, rules4 = ruleGenerator(data4, 0.1, float(1))
# items5, rules5 = ruleGenerator(data_5, 0.2, float(1))

In [106]:
def ruleGen(items, rules1):
    sets1 = []
    sets2 = []
    dfListUnion = []
    dfListIntersection = []
    sets1.append(set(rules1))
    sets2.append(set(items))

    union = set.union(*sets1)
    lUnion = list(union)

    for rul in range(0, len(lUnion)):
                dfAux_union = {'LHS': tuple(lUnion[rul].lhs), 'LHS_size': len(lUnion[rul].lhs),
                            'RHS': tuple(lUnion[rul].rhs), 'RHS_size': len(lUnion[rul].rhs),
                            'conf': lUnion[rul].confidence,
                            'sup': lUnion[rul].support, 'lift': lUnion[rul].lift, 'cov': lUnion[rul].conviction}
                dfListUnion.append(dfAux_union)

    dfRule = pd.DataFrame(dfListUnion)

    return dfRule

In [107]:
dfRule1 = ruleGen(items1, rules1)
dfRule2 = ruleGen(items2, rules2)
dfRule3 = ruleGen(items3, rules3)
dfRule4 = ruleGen(items4, rules4)
# dfRule5 = ruleGen(items5, rules5)

In [108]:
dfRule1.to_csv('rules_data1.csv')
dfRule2.to_csv('rules_data2.csv')
dfRule3.to_csv('rules_data3.csv')
dfRule4.to_csv('rules_data4.csv')

In [109]:
dfRule1

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_a == 2,)",1,"(MR1_input_b == 2.0,)",1,1.0,0.1,10.0,900000000.0
1,"(MR1_input_b == 2.0,)",1,"(input_a == 2,)",1,1.0,0.1,10.0,900000000.0
2,"(input_a == 6,)",1,"(MR1_input_b == 6.0,)",1,1.0,0.1,10.0,900000000.0
3,"(MR1_input_b == 6.0,)",1,"(input_a == 6,)",1,1.0,0.1,10.0,900000000.0
4,"(input_a == 8,)",1,"(MR1_input_b == 8.0,)",1,1.0,0.1,10.0,900000000.0
5,"(MR1_input_b == 8.0,)",1,"(input_a == 8,)",1,1.0,0.1,10.0,900000000.0
6,"(input_b == 8,)",1,"(MR1_input_a == 8.0,)",1,1.0,0.1,10.0,900000000.0
7,"(output == 0.0,)",1,"(MR1_checker == No-violated,)",1,1.0,0.1,10.0,900000000.0
8,"(MR1_checker == No-violated,)",1,"(output == 0.0,)",1,1.0,0.1,10.0,900000000.0
9,"(input_b == 0,)",1,"(MR1_input_a == 0.0,)",1,1.0,0.1,10.0,900000000.0


In [110]:
dfRule2

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_a == 2,)",1,"(MR1_input_b == 2.0,)",1,1.0,0.1,10.0,900000000.0
1,"(MR1_input_b == 2.0,)",1,"(input_a == 2,)",1,1.0,0.1,10.0,900000000.0
2,"(input_a == 6,)",1,"(MR1_input_b == 6.0,)",1,1.0,0.1,10.0,900000000.0
3,"(MR1_input_b == 6.0,)",1,"(input_a == 6,)",1,1.0,0.1,10.0,900000000.0
4,"(input_a == 8,)",1,"(MR1_input_b == 8.0,)",1,1.0,0.1,10.0,900000000.0
5,"(MR1_input_b == 8.0,)",1,"(input_a == 8,)",1,1.0,0.1,10.0,900000000.0
6,"(input_b == 8,)",1,"(MR1_input_a == 8.0,)",1,1.0,0.1,10.0,900000000.0
7,"(output == 0.0,)",1,"(MR1_checker == No-violated,)",1,1.0,0.1,10.0,900000000.0
8,"(MR1_checker == No-violated,)",1,"(output == 0.0,)",1,1.0,0.1,10.0,900000000.0
9,"(input_b == 0,)",1,"(MR1_input_a == 0.0,)",1,1.0,0.1,10.0,900000000.0


In [111]:
dfRule3

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_a == 2,)",1,"(MR1_input_b == 2.0,)",1,1.0,0.1,10.0,900000000.0
1,"(MR1_input_b == 2.0,)",1,"(input_a == 2,)",1,1.0,0.1,10.0,900000000.0
2,"(input_a == 6,)",1,"(MR1_input_b == 6.0,)",1,1.0,0.1,10.0,900000000.0
3,"(MR1_input_b == 6.0,)",1,"(input_a == 6,)",1,1.0,0.1,10.0,900000000.0
4,"(input_a == 8,)",1,"(MR1_input_b == 8.0,)",1,1.0,0.1,10.0,900000000.0
5,"(MR1_input_b == 8.0,)",1,"(input_a == 8,)",1,1.0,0.1,10.0,900000000.0
6,"(input_b == 8,)",1,"(MR1_input_a == 8.0,)",1,1.0,0.1,10.0,900000000.0
7,"(output == 0.0,)",1,"(MR1_checker == No-violated,)",1,1.0,0.1,10.0,900000000.0
8,"(MR1_checker == No-violated,)",1,"(output == 0.0,)",1,1.0,0.1,10.0,900000000.0
9,"(input_b == 0,)",1,"(MR1_input_a == 0.0,)",1,1.0,0.1,10.0,900000000.0


In [112]:
dfRule4

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_a == 2,)",1,"(MR1_input_b == 2.0,)",1,1.0,0.1,10.0,900000000.0
1,"(MR1_input_b == 2.0,)",1,"(input_a == 2,)",1,1.0,0.1,10.0,900000000.0
2,"(input_a == 6,)",1,"(MR1_input_b == 6.0,)",1,1.0,0.1,10.0,900000000.0
3,"(MR1_input_b == 6.0,)",1,"(input_a == 6,)",1,1.0,0.1,10.0,900000000.0
4,"(input_a == 8,)",1,"(MR1_input_b == 8.0,)",1,1.0,0.1,10.0,900000000.0
5,"(MR1_input_b == 8.0,)",1,"(input_a == 8,)",1,1.0,0.1,10.0,900000000.0
6,"(MR1_input_a == 8.0,)",1,"(input_b == 8,)",1,1.0,0.1,10.0,900000000.0
7,"(input_b == 8,)",1,"(MR1_input_a == 8.0,)",1,1.0,0.1,10.0,900000000.0
8,"(input_a == 1,)",1,"(MR1_input_b == 1.0,)",1,1.0,0.1,10.0,900000000.0
9,"(input_b == 0,)",1,"(MR1_input_a == 0.0,)",1,1.0,0.1,10.0,900000000.0
